In [1]:
import pandas as pd
from pathlib import Path

import sys
sys.path.insert(0, '../..')
from src.utils.latex import show_latex, TABLES

# Load data

In [2]:
datapath = Path('../../data/2017_raw')

notes = pd.read_pickle(datapath / 'processed.pkl')
annotated = pd.read_csv('../../data/annotated_notes_ids.csv', dtype={'MDN': str, 'NotitieID': str})

# Stats

In [3]:
caption = "All data: num patients, num notes"
label = "2017_all_overview"

pd.concat([
    notes.groupby('institution').MDN.nunique().rename('patients'),
    notes.groupby('institution').NotitieID.nunique().rename('notes'),
    ], axis=1
).T.assign(total=lambda df: df.sum(axis=1)).rename_axis('n_unique').pipe(show_latex, caption, label)

institution,amc,vumc,total
n_unique,,,
patients,"99,867","79,970","179,837"
notes,"2,375,627","1,869,079","4,244,706"


In [4]:
caption = "Overview annotated data"
label = "2017_annot"

selection = annotated.query("year==2017")
piv = pd.concat([
    selection.groupby('institution')[['NotitieID']].nunique(),
    notes.groupby('institution')[['NotitieID']].nunique(),
], keys=['annotated', 'all']).rename(columns={'NotitieID':'notes'}
).unstack(0).stack(0).fillna(0).astype(int
).assign(rest=lambda df: df['all'] - df.annotated
).reindex(['all', 'annotated', 'rest'], axis=1)

piv.append(pd.concat([piv.xs('amc') + piv.xs('vumc')], keys=['total'])).pipe(show_latex, caption, label)

,,all,annotated,rest
institution,,,,
amc,notes,"2,375,627",0,"2,375,627"
vumc,notes,"1,869,079","3,377","1,865,702"
total,notes,"4,244,706","3,377","4,241,329"


In [5]:
prefix = '2017'
for idx, table in enumerate(TABLES):
    with open(f'./tables/{prefix}_{idx}.tex', 'w', encoding='utf8') as f:
        f.write(table)